In [1]:
import bugzilla
import requests
import time
from datetime import datetime

In [2]:
pagure_owner_alias = requests.get("https://src.fedoraproject.org/extras/pagure_owner_alias.json").json()["rpms"]

In [3]:
orphaned = [p for p in pagure_owner_alias if "orphan" in pagure_owner_alias[p]]
orphaned[:10]

['8Kingdoms',
 '8sync',
 '915resolution',
 '99soft-oss-parent',
 'AcetoneISO2',
 'Ajaxterm',
 'CPUFreqUtility',
 'CableSwig',
 'Canna',
 'CastPodder']

In [4]:
bzapi = bugzilla.Bugzilla("bugzilla.redhat.com")

In [5]:
TRACKER = 1927309

query = bzapi.build_query(product="Fedora", include_fields=["flags", "assigned_to", "component", "creation_time"])
query["blocks"] = TRACKER
query["status"] = "NEW"
query["limit"] = 1000
bugzillas = bzapi.query(query)
len(bugzillas)

59

In [6]:
RELENG = "releng@fedoraproject.org"
ORPHAN = "extras-orphan@fedoraproject.org"

In [7]:
pkgs_to_orphan = []
bugs_to_reassign = []

In [8]:
for bug in bugzillas:
    if bug.component in orphaned:
        #if bug.assigned_to != ORPHAN:
        #    bugs_to_reassign.append(bug.id)
        continue
    needinfos = bug.get_flags("needinfo") or []
    for needinfo in needinfos:
        if needinfo.get('requestee') == bug.assigned_to:
            break
    else:
        # no needinfo on assignee
        continue
    bug_state = 0
    last_time = datetime.fromtimestamp(time.mktime(bug.creation_time.timetuple()))
    for comment in bug.getcomments():
        if comment["creator"] != RELENG:
            continue
        if not comment["text"].startswith("Dear Maintainer,"):
            continue
        comment_time = datetime.fromtimestamp(time.mktime(comment["creation_time"].timetuple()))
        delta = comment_time - last_time
        if bug_state == 0 and delta.days >= 7-1:
            last_time = comment_time
            bug_state = 1
        elif bug_state == 1 and delta.days >= 3*7-1:
            last_time = comment_time
            bug_state = 2
            break
    delta = datetime.now() - last_time
    if bug_state == 2 and delta.days >= 4*7-1:
        to_orphan = f"rpms/{bug.component}"
        pkgs_to_orphan.append(to_orphan)
        #print(f"    '{to_orphan}': 'https://bugzilla.redhat.com/{bug.id}',")
        print(to_orphan)
        bugs_to_reassign.append(bug.id)

rpms/erlang-riak_api
rpms/erlang-riak_core
rpms/forbidden-apis
rpms/libcxl


In [9]:
print("https://bugzilla.redhat.com/buglist.cgi?bug_id=" + ",".join(str(b) for b in bugs_to_reassign))

https://bugzilla.redhat.com/buglist.cgi?bug_id=1987459,1987460,1987484,1987643
